#### Defining root directory and importing packages (Cell 1):

In [ ]:
# Cell 1: 

root_dir = r'C:/your/path/to/lfmSPTanalysis-main' # set this as your absolute path to lfmSPTanalysis-main


from my_package import fitting_2D, fitting_3D, optimising_3DFit, processing_3Dlocs, smTracking, analysingTracks, summarise_analysis, resplot, copy_project
import os, matplotlib.pyplot as plt

## 2D Fitting using ImageJ GDSC PeakFit (Cell 2): 

In [ ]:
# Cell 2:
# Refer to README.md section 2.2 for more information about running this cell.

source_path = r"C:/path/to/your/input/file/or/folder" # absolute path to your input file or folder. 

runLoop = False # set to False if your source path is 1) a single image stack OR 2) a single directory containing multiple image stacks of the same FOV.
                # set to True if your source path is a folder containing multiple image stacks of different FOVs, all to be analysed in one go.

config_name = 'fitting2D_config.json' # name of the config file from the 'configs' folder to be used for 2D fitting. Include '.json'
# can change 2D fitting settings (including PeakFit parameters) by opening and modifying the 'configs/fitting2D_config.json' file.

fiji_path = r'C:\Users\username\Applications\Fiji.app'  # path to your Fiji.app installation from README.md step 1.2


if runLoop:
    fitting_2D.gdsc_peakFit(source_path, runLoop, fiji_path, root_dir, config_name) # returns nothing
else:
    return_locs2D_path, mc = fitting_2D.gdsc_peakFit(source_path, runLoop, fiji_path, root_dir, config_name)

## 3D Fitting using PySMLFM (Cell 3):

In [ ]:
# Cell 3:
# Refer to README.md section 2.3 for more information about running this cell. Credits to https://github.com/Photometrics/PySMLFM for the 3D fitting package.

runLoop = False # set to False to run this for a single 2D localisation csv file. If false, then define locs2D_path accordingly. 
                # set to True to batch process all 2D localisation csv files in the 'data/2D_locs_csv/' folder.

locs2D_path = return_locs2D_path # set as locs2D csv path returned from Cell 2.
# if locs2D_path from cell 2 is lost, set this as the abosulte path of your 2D localisation csv file from the 'data/2D_locs_csv/' folder, e.g.:
# locs2D_path = r'C:/your/path/to/lfmSPTanalysis-main/data/2D_locs_csv/your_locs2D.csv'

cfg_name = 'fitting3D_config.json' # name of the config file from the 'configs' folder to be used for 3D fitting. Include '.json'
# can change 3D fitting settings by opening and modifying the file within the configs folder.


plt.close('all')
%matplotlib inline
if runLoop:
    fitting_3D.batchFittingAxialLocs(root_dir, cfg_name) # returns nothing
else:
    return_locs3D_path = fitting_3D.fittingAxialLocs(locs2D_path, root_dir, cfg_name)[0] 

### Optimising MLA alignment for 3D Fitting (Cell 4):

In [ ]:
# Cell 4: 
# Refer to README.md section 2.4 for more information about running this cell.

locs2D_path = r'C:/path/to/lfmSPTanalysis-main/data/2D_locs_csv/file.csv' # absolute path to the 2D localisation csv file to be used for optimising 3D fitting alignment.

base_cfg_name = 'fitting3D_config.json' # name of the config file from the 'configs' folder to be used as a base for the optimisation. Include '.json'
# ensure that this base config file has the 'show_graphs' parameter set to False. 

# defining search bounds (min, max) for the parameters to be optimised. offset limits in microns.
rotation_search = (-5,5)
x_offset_search = (-10,10) 
y_offset_search = (-5,15)

# params for running gp_minimize() - bayesian optimisation using a gaussian process.
n_calls = 150
n_initial_points = 30



optimising_3DFit.optimise_MLA_alignment(root_dir, [rotation_search,x_offset_search,y_offset_search], locs2D_path, base_cfg_name, n_calls = n_calls, n_initial_points=n_initial_points)

## Plotting 3D Localisations (Cell 5):

In [ ]:
# Cell 5 (Plotting scatter 3D locs for qualitative inspection):
# Refer to README.md section 2.5 for more information about running this cell.

fm_locs3D_path = return_locs3D_path # set as locs3D csv path returned from Cell 3.
# alternatively set this as the abosulte path of your 3D localisation csv file from the 'data/formatted_3Dlocs/' folder, e.g.:
# fm_locs3D_path = r'C:/your/path/to/lfmSPTanalysis-main/data/formatted_3Dlocs/your_locs3D_formatted.csv'



%matplotlib qt
data_bounds, fm_path = processing_3Dlocs.plot_3Dlocs(fm_locs3D_path)


## Cropping 3D Localisations (Cell 6):

In [ ]:
# Cell 6 (cropping 3D localisations):
# Refer to README.md section 2.6 for more information about running this cell.

# set these as your cropping boundaries (in microns) for the dataset plotted from cell 6.
# if no cropping needed for the upper or lower bound of a certain axis, set as None.
x_min = None
x_max = None
y_min = None
y_max = None
z_min = 2
z_max = None



processing_3Dlocs.crop_3Dlocs(root_dir, fm_path, data_bounds, [x_min,x_max,y_min,y_max,z_min,z_max])

## Single particle tracking (cell 7):

In [ ]:
# Cell 7:
# Refer to README.md section 2.7 for more information about inputs. credits to https://github.com/wb104/trajectory-analysis for tracking code.

# Runs analysis for all cropped_locs3D.csv files from the 'data/cropped_3Dlocs' folder.

smTracking.sm_tracking(root_dir)

## Analysing tracking data via MATLAB (cell 8):

In [ ]:
# Cell 8:
# Refer to the README.md section 2.8 for more information.

# to edit parameters, open the "\src\AnalyzeTrackingData_withDirection_master.m" script, edit, and save.
# running this cell will input all csv files in the "\data\tracks" directory so ensure it only contains track files for a single sample/condition.

analysingTracks.matlab_trackingDataAnalysis(root_dir)
# Output: an output folder within the 'results/' directory named after the data & time of the analysis.


## Plotting distributions of TF biophysical parameters + summarising analysis (cells 9-10):

##### Single Sample/Condition (cell 9):

In [ ]:
# Cell 9:
# Generating summary of analysis (data processing) for a single sample type. 
# Generating per FOV distibution plots for TF biophysical parameters for a single sample type.

results_dir = r'path/to/lfmSPTanalysis-main/results/matlab_output_directory/' # absolute path to the matlab output directory generated from Cell 8.

sample_name = 'sample/condition_name' # name of the sample type you are analysing.

%matplotlib qt
summarise_analysis.compile_stats(results_dir, sample_name, root_dir)
resplot.batchProcess_jitterSingle(results_dir, sample_name, root_dir)

##### Comparing Samples/Conditions (cell 10):

In [ ]:
# Cell 10:
# Generating per FOV distibution plots for TF biophysical parameters for n sample types and statistically comparing them. e.g. n=3:

results_dirs = [r'path/to/lfmSPTanalysis-main/results/matlab_output_directory_of_sample1/', 
                r'path/to/lfmSPTanalysis-main/results/matlab_output_directory_of_sample2/',
                r'path/to/lfmSPTanalysis-main/results/matlab_output_directory_of_sample3/']
# list of absolute paths to the results directories generated from Cell 8 for the sample types you are comparing.

summary_csvs = [r'path/to/lfmSPTanalysis-main/results/sample1_name/analysisSummary/sample1_name_stats_summary.csv', 
                r'path/to/lfmSPTanalysis-main/results/sample1_name/analysisSummary/sample2_name_stats_summary.csv',
                r'path/to/lfmSPTanalysis-main/results/sample1_name/analysisSummary/sample3_name_stats_summary.csv']
# list of absolute paths to the stats csv file generated from Cell 9 for the sample types you are comparing.

sample_names = ['sample1_name', 'sample2_name', 'sample3_name'] # list of names of the sample types you are comparing in the same order as the list of results_dirs provided.

%matplotlib qt
summarise_analysis.jitter_pipelineStats(summary_csvs, sample_names, root_dir)
resplot.batchProcess_jitterMulti_sx(results_dirs, sample_names, root_dir)
resplot.plotAssociationValues(results_dirs, sample_names, root_dir)

## Saving data and results (cell 11):

In [ ]:
# Cell 11:

# copy the 'data' and 'results' folders to a destination folder. can then empty workspace and start new project.

destination_dir = '' # provide path to destination folder
copy_project(root_dir, destination_dir)